In [1]:
# sudo xboxdrv --silent

In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
import signal
from xbox360controller import Xbox360Controller
import json
from websocket import create_connection

def send_cmd(command, *args, **kwargs):
    req = dict(command=command,
               args=args,
               kwargs=kwargs)
    data = json.dumps(req)
    ws = create_connection("ws://localhost:9001")
    ws.send(data)
    res = json.loads(ws.recv())
    ws.close()
    return res

In [4]:
import ipywidgets as widgets
from IPython.display import clear_output, display
import numpy as np

def move_delta(move):
    send_cmd("move_delta", **move)

def move_on_button_press(slider):
    def inner(button):
        to_axis = {"y": "+z",         "a": "-z"
                #   , "x": "-y",         "b": "-y"
                #   , "trigger_r": "+z", "trigger_l": "-z"
                  }
        if (cmd := to_axis.get(button.name[7:])) is None:
            return
        sign, axis = {"-": -1, "+": 1}[cmd[0]], cmd[1]
        
        move = dict(x=0, y=0, z=0) | {axis : sign * slider.value}
        move_delta(move)
    return inner
    
def process_hat(slider):
    def inner(axis):
        move = dict( x=-axis.y * slider.value
                   , y=axis.x * slider.value
                   , z=0 )
        move_delta(move)
    return inner

def scale_changer():
    slider = widgets.FloatSlider(min=0.1, max=20)
    steps = [.1, 1, 3, 6, 10, 15, 20]
    cur_step = 3
    slider.value = steps[cur_step]
    
    def change(button):
        nonlocal cur_step
        cur_step +=( { "trigger_r": +1
                     , "trigger_l": -1
                     } 
                     [button.name[7:]] )
        cur_step = np.clip(cur_step, 0, len(steps) - 1)
        slider.value = steps[cur_step]
    
    display(slider)
    return change, slider


In [5]:
try:
    with Xbox360Controller(0, axis_threshold=0.2) as controller:
        # Button A events
        changer, slider = scale_changer()
        controller.hat.when_moved = process_hat(slider)
        for button in controller.buttons:
            if "trigger" not in button.name:
                button.when_pressed = move_on_button_press(slider)
            else:
                button.when_pressed = changer
        signal.pause()
except KeyboardInterrupt:
    pass

this gamepad or its driver.


FloatSlider(value=6.0, max=20.0, min=0.1)